# Part 3

Reference - https://m.blog.naver.com/samsjang/220985170721

In [ ]:
import re
import pandas as pd 
from time import time

def preprocessor(text) :
    text = re.sub('<[^>]*>', '',text)
    emoticons = re.findall('?::|;|=)(?:-)?(?:\)|\(|D|P)|\^.?\^', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-','')
    
    return text

df = pd.read_csv('./labeledTrainData.tsv',header = 0, delimiter='\t', quoting =3)

stime = time()
df['review'] = df['review'].apply(preprocessor)

In [53]:
pwd

'C:\\Users\\kjh96\\OneDrive\\Desktop\\Github\\kaggle\\IMDB_movie_review\\data'

In [54]:
cd C://Users//kjh96//OneDrive//Desktop//Github//kaggle/IMDB_movie_review/data

C:\Users\kjh96\OneDrive\Desktop\Github\kaggle\IMDB_movie_review\data


In [55]:
df = pd.read_csv('./labeledTrainData.tsv',header = 0, delimiter='\t', quoting =3)

In [56]:
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [57]:
from bs4 import BeautifulSoup
for i in range(len(df)) :
    df.review[i] = BeautifulSoup(df.review[i]).get_text()
    df.review[i] = re.sub("[^a-zA-Z]"," ", df.review[i])
    df.review[i] = df.review[i].lower()

C:\Users\kjh96\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kjh96\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\kjh96\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [58]:
df.head()

,id,sentiment,review
0,"""5814_8""",1,with all this stuff going down at the moment ...
1,"""2381_9""",1,the classic war of the worlds by timothy ...
2,"""7759_3""",0,the film starts with a manager nicholas bell...
3,"""3630_4""",0,it must be assumed that those who praised thi...
4,"""9495_8""",1,superbly trashy and wondrously unpretentious ...


In [59]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

porter = PorterStemmer()
stop = stopwords.words('english')

def tokenizer(text) :
    return text.split()

def tokenizer_porter(text) :
    return [porter.stem(word) for word in text.split()]

In [60]:
text = 'runners like running and thus run'
print(tokenizer(text))
print(tokenizer_porter(text))

['runners', 'like', 'running', 'and', 'thus', 'run']
['runner', 'like', 'run', 'and', 'thu', 'run']


In [63]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import pickle
import os
from time import time
import pandas as pd

In [64]:
X_train = df.loc[:35000, 'review'].values
y_train = df.loc[:35000, 'sentiment'].values
X_test = df.loc[15000:,'review'].values
y_test = df.loc[15000:,'review'].values

In [65]:
tfidf = TfidfVectorizer(lowercase=False, tokenizer = tokenizer)
lr_tfidf = Pipeline([('vect', tfidf), ('clf', LogisticRegression(C=10.0, penalty = '12', random_state = 0))])

In [ ]:
stime = time()
print('머신러닝 시작')
lr_tfidf.fit(X_train, y_train)
print('머신러닝 종료')

y_pred = lr_tfidf.predict(X_test)

In [ ]:
print('테스트 종료 : 소요시간 [%d]초' %(time()-stime))
print('정확도 : %.3f' %accuracy_score(y_test, y_pred))